# Giris
Merhaba arkadaşlar, bu çalışmamızda Keras Tuner üzerinde çalışmalar yapacağız. Keras Tuner, TensorFlow programımız için en uygun hiperparametre kümesini seçmenize yardımcı olan bir kütüphanedir.<br>
Hiperparametre, bir ML modelinin eğitim sürecini ve topolojisini yöneten değişkenlerdir.<br>

Hiperparametreler iki tiptir:
- Model hiperparametreleri, gizli katmanların (Hidden layers) sayısı, nöron sayısı, aktivasyon (activation) fonksiyonu, giriş şekli (input shape) vb. kavramlar.

- Stochastic Gradient Descent (SGD) için öğrenme oranı ve k Nearest Neighbors  (KNN) sınıflandırıcısı için en yakın komşu sayısı gibi öğrenme algoritmasının hızını ve kalitesini etkileyen algoritma hiperparametreleri

In [1]:
# Kullanacagimzi Kutuphaneler
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

# Veri Seti Aşamaları
Bu bölümde, veri setini indirecek ve eğitim yapacağımız modeller için veri setimizi düzenleyeceğiz.

In [4]:
# Veri setinin indirilmesi ve train, test veri setileri olarak ayrilmasi
(train_images, train_labels),(test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Piksel veri setirinden olusan image veri setilerinin icerisideki degerleri 'float32' cevirip 255.0 bolerek (0-255) arasinda olan degerleri (0-1) araligina cekiyoruz
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.

In [8]:
# Egitim veri kumesinin sekli
train_images[0].shape

(28, 28)

# Modelin Tanmlanması
Bu bölümde, modelimizin tasarımını Keras Tuner kullanarak yapacağız.

In [11]:
def model_builder(hp):
  model = keras.Sequential()
  # Bu seger input_shape de bir degisiklik yaptim. Direk train veri kumesini sayi ile yazmak yerine .shape kullanarak yazdim. Aslinda asagida yazan input_shape = (28, 28) aynisidir.
  model.add(keras.layers.Flatten(input_shape = (train_images[0].shape[0], train_images[0].shape[1])))

  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32) # Dense katmaninda noron sayisini belirliyoruz.
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) # Learning_rate ogrenme oranini belirliyoruz.

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])

  return model

Kod hakkinda kisa bilgiler:
- hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32): Bu satır, modelin dense katmanındaki nöron sayısını belirler. Keras Tuner, 32 ile 512 arasında bir değer seçecek ve her seferinde 32 artarak denemeler yapacaktır (örneğin: 32, 64, 96, ... 512).
- hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]): Bu satır, öğrenme oranını seçmek için Keras Tuner’a üç olasılık sunar: 0.01, 0.001 ve 0.0001. Keras Tuner bu seçeneklerden birini seçecek ve modeli farklı öğrenme oranlarıyla eğitecektir.

# Tuner Optimizasyonu
Keras Tuner, model hiperparametre ayarlarını yapmak için kullanılan bir araçtır. Hiperparametreler, modelin eğitimi sırasında kullanıcı tarafından belirlenen ve modelin performansını doğrudan etkileyen parametrelerdir.

Keras Tuner'da Mevcut Dort Farkli Tuner vardir:
1. RandomSearch (Rastgele Arama):
  - Bu yöntem, belirli bir aralıkta rastgele hiperparametreler seçer ve bu hiperparametre kombinasyonlarının her birini değerlendirir. Bu, genellikle basit ama etkili bir tekniktir.
  - Avantajı: Uygulaması kolaydır ve genellikle küçük hesaplama kaynaklarına sahip projeler için uygundur.
  - Dezavantajı: Bazen çok büyük hiperparametre aralıklarında optimize etme süresi uzun olabilir.
2. Hyperband:
  - Hyperband, daha gelişmiş ve verimli bir arama yöntemidir. Bu tuner, çok sayıda denemeyi hızlı bir şekilde test eder ve performansı düşük olanları erken bir aşamada terk eder. Kalan denemeler üzerine daha fazla kaynak ayırarak en iyi hiperparametreyi bulmayı hedefler.
  - Avantajı: Kaynakları daha verimli kullanır ve hızlı sonuçlar alabilirsiniz.
  - Dezavantajı: Büyük arama alanlarında bile daha iyi performans gösterse de, çok fazla hesaplama kaynağı gerektirebilir.
3. BayesianOptimization (Bayezyen Optimizasyonu):
  - Bayezyen Optimizasyonu, olasılık teorisi ve istatistiksel modelleri kullanarak hiperparametreleri optimize eder. Bu yöntem, en uygun hiperparametreleri bulmak için geçmiş denemelerden edindiği bilgiye dayanarak daha akıllıca seçimler yapar.
  - Avantajı: Daha az deneme yaparak daha iyi sonuçlar alabilirsiniz. Geçmiş deneyimleri dikkate alarak daha verimli bir şekilde öğrenir.
  - Dezavantajı: Diğer yöntemlere göre daha karmaşıktır ve bazı durumlarda daha fazla hesaplama kaynağı gerektirebilir.
4. klearn (Scikit-Learn):
  - Sklearn, klasik makine öğrenmesi algoritmalarını kullanan bir optimizasyon yöntemidir. Scikit-learn ile uyumlu olan bu tuner, hiperparametre arama sürecini makine öğrenmesi yöntemlerine dayandırarak gerçekleştirebilir.
  - Avantajı: Klasik makine öğrenmesi modelleri için idealdir ve hızlıdır.
  - Dezavantajı: Derin öğrenme gibi karmaşık modellerde diğer yöntemler kadar etkili olmayabilir.

Bu aşamada Hyperband'i kullanacağız.

Not: Hyperband tuner'ı örneklendirmek için, hipermodeli, optimize edilecek hedefi ve eğitilecek maksimum dönem sayısını (max_epochs) belirtmeniz gerekir.

In [12]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Hyperband ayarlama algoritması, yüksek performanslı bir modele hızla ulaşmak için uyarlanabilir kaynak tahsisi ve erken durdurmayı kullanır. Algoritma, birkaç dönem için çok sayıda modeli eğitiyor ve modellerin yalnızca en iyi performans gösteren yarısını bir sonraki tura taşıyor. Hyperband, 1 + logfactor(max_epochs) değerini hesaplayarak ve en yakın tam sayıya yuvarlayarak bir parantez içinde eğitilecek model sayısını belirler.

In [13]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

Yukarıdakı kod, Keras'ta early stopping (erken durdurma) özelliğini tanımlar. Early stopping, modelin eğitim sürecinde belirli bir noktada performansı kötüleşmeye başladığında eğitimi erken durdurmak için kullanılır

val_loss değeri 5 ardışık adım (epoch) sonunda iyileşmezse, model eğitimi durdurulur.

In [14]:
tuner.search(train_images, train_labels, epochs = 50, validation_split = 0.2, callbacks = [stop_early])

Trial 30 Complete [00h 02m 43s]
val_accuracy: 0.8860833048820496

Best val_accuracy So Far: 0.8884166479110718
Total elapsed time: 00h 25m 12s


In [17]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"En optimal noron sayisi: {best_hps.get('units')}, En optimal ogrenme orani: {best_hps.get('learning_rate')}")

En optimal noron sayisi: 160, En optimal ogrenme orani: 0.001


30 eğitim tamamladık ve en iyi val_accuracy (doğrulama veri kümesi ile bulunan kesinlik değeri) 0.88'dir. Bu süreç toplam 25 dakika sürmüştür. En optimal 'units' (dense layer içinde nöron sayısı) 160 olarak saptanmış ve en iyi learning_rate (öğrenme oranımız) 0.001 olarak bulunmuştur.

# Model Eğitimi
Bu bölümde, Tuner fonksiyonu ile bulduğumuz en iyi parametreleri kullanarak modelimizi optimize edeceğiz.

In [19]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs = 50, validation_split = 0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f"En iyi epoch: {best_epoch}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7738 - loss: 0.6570 - val_accuracy: 0.8188 - val_loss: 0.4947
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8599 - loss: 0.3898 - val_accuracy: 0.8602 - val_loss: 0.3856
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8704 - loss: 0.3549 - val_accuracy: 0.8748 - val_loss: 0.3464
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8842 - loss: 0.3163 - val_accuracy: 0.8533 - val_loss: 0.3957
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8929 - loss: 0.2959 - val_accuracy: 0.8815 - val_loss: 0.3331
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8967 - loss: 0.2805 - val_accuracy: 0.8809 - val_loss: 0.3330
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8983 - loss: 0.2684 - val_accuracy: 0.8874 - val_loss: 0.3201
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9071 - loss: 0.2511 

In [20]:
# En iyi epoch degerini kullanarak yeni bir egitim gerceklestirecegiz
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(train_images, train_labels, epochs = best_epoch, validation_split = 0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7759 - loss: 0.6416 - val_accuracy: 0.8477 - val_loss: 0.4201
Epoch 2/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8578 - loss: 0.3956 - val_accuracy: 0.8717 - val_loss: 0.3617
Epoch 3/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8765 - loss: 0.3437 - val_accuracy: 0.8687 - val_loss: 0.3664
Epoch 4/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.8841 - loss: 0.3140 - val_accuracy: 0.8811 - val_loss: 0.3330
Epoch 5/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8883 - loss: 0.3012 - val_accuracy: 0.8785 - val_loss: 0.3385
Epoch 6/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8958 - loss: 0.2859 - val_accuracy: 0.8737 - val_loss: 0.3485
Epoch 7/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9004 - loss: 0.2659 - val_accuracy: 0.8842 - val_loss: 0.3317
Epoch 8/22
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9050 - loss: 0.25

In [21]:
eval_result = hypermodel.evaluate(test_images, test_labels)
print(f"[test loss(kayip), test accuracy(kesinlik)]: ", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8854 - loss: 0.3768
[test loss(kayip), test accuracy(kesinlik)]:  [0.38698822259902954, 0.8815000057220459]


# Sonuc
Introduction to the Keras Tuner (Keras Tuner'a giriş) bölümünün sonuna geldik. Bu çalışmada, Keras Tuner fonksiyonunu nasıl uygulayacağımızdan bahsettik. Umarım çalışmam sizin için faydalı olmuştur. İyi günler, iyi çalışmalar dilerim.

Aşağıdaki Linklerden beni takip edebilir ve yapacağım çalışmalardan haberdar olabilirsiniz!<br>
[Linkedin](https://www.linkedin.com/in/ihsancenkiz/)<br>
[Github](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)